# Analyse Badlands stratigraphic output

If the stratigraphic structure is turned on in the XmL input file, **Badlands** produces sedimentary layers Hdf5 files. The stratigraphic layers are defined on a regularly spaced grid and a layer is recorded at each layer time interval given by the user.

Here we show how we can visualise quickly the structure of the stratigraphic layer in an IPython notebook.

In [ ]:
%matplotlib inline

import numpy as np
import colorlover as cl

# Import badlands grid generation toolbox
import pybadlands_companion.stratalAnalyse as strata

# display plots in SVG format
%config InlineBackend.figure_format = 'svg' 

# Loading stratigraphic file

First we need to load one of the stratigraphic file. The files are located in the **h5/** folder in the simulation main output folder and are named using the following convention:
- `sed.time`T`.p`X`.hdf5`

with T the display time index and X the number of the partition (used in the parallel version). In cases where you ran your simulation in parallel you will also need to give the number of CPUs used (_cpus_).

To load a file you will need to give the folder name and the number of processors used in your simulation.

For more information regarding the function uncomment the following line.

In [ ]:
#help(strata.stratalSection.__init__)

In [ ]:
folder = '/workspace/volume/pyBadlands/Examples/delta/output/h5/'
strat = strata.stratalSection(folder,1)

Then we need to load a particular output time interval (this is the T parameter in the hdf5 file name convention).

**Note**

This number is not always the number of sedimentary layers for this particular time step as you could have chosen in the input file to have more than 1 sedimentary layer recorded by output interval!

In [ ]:
#help(strat.loadStratigraphy)

In [ ]:
strat.loadStratigraphy(0,50)

# Building a cross-section

We then slice the stratigraphic mesh to visualise the sedimentary architecture along a given cross-section.

To create the cross-section you will need to provide the position of the segment in the simulation space _(xo,yo)_ and _(xm,ym)_, a gaussian filter value for smoothing (_gfilt_ a value of 0 can be used for non-smoothing) and the resolution of the cross-section (based on a number of points: _nbpts_). 

In [ ]:
# Position of the cross-section
x1 = 10000
y1 = strat.y.min()
x2 = 10000
y2 = strat.y.max()

#<llcx>87000</llcx>
#<llcy>34000</llcy>
#<urcx>132000</urcx>
#<urcy>169000</urcy>
            
# Interpolation parameters
gfilt = 5
nbpts = 1000

In [ ]:
#help(strat.buildSection)

In [ ]:
strat.buildSection(xo = x1, yo = y1, xm = x2, ym = y2, pts = nbpts, gfilter = gfilt)

# Plot cross-section

We use **plotly** to visualise the cross-section and the associated sedimentary packages.

In [ ]:
#help(strata.viewSection)

In [ ]:
strata.viewSection(width = 1000, height = 600, cs=strat, 
                   rangeX=[0,strat.dist.max()], rangeY=[-1500,500], 
                   linesize=0.5, title='stratigraphic stack coloured by time')

# System tract

There are several models of systems tracts within depositional sequences, here we use the most simple one called the four systems tract model:

- lowstand systems tract **LST**, 
- transgressive systems tract **TST**, 
- highstand systems tract **HST**, and 
- falling-stage systems tract **FST**.

For each of these system tracts we define a given color. We use the 'colorlover' library [link](http://moderndata.plot.ly/color-scales-in-ipython-notebook/)

In [ ]:
#colormap = cl.scales['9']['seq']['BuPu']
#colorrgb = cl.to_rgb( colormap )
#cLST = colormap[1]
#cTST = colormap[3]
#cHST = colormap[5]
#cFST = colormap[7]

cLST = 'rgb(220,213,166)' #colormap[1]
cTST = 'rgb(161,206,146)' #colormap[3]
cHST = 'rgb(72, 106,162)' #colormap[5]
cFST = 'rgb(119,11,116)' #colormap[7]

### Plot sea-level fluctuations

In [ ]:
seafile = '/workspace/volume/pyBadlands/Examples/delta/data/sea.csv'
stime,sea = strata.viewSea(seafile)

Based on the sea-level curve, we define a series of **system tracts** periods.

In [ ]:
LST1 = np.array([0,90000],dtype=int)
TST1 = np.array([90000,160000],dtype=int)
HST1 = np.array([160000,220000],dtype=int)
FST1 = np.array([220000,340000],dtype=int)
LST2 = np.array([340000,420000],dtype=int)
TST2 = np.array([420000,500000],dtype=int)

The simulation ran for 500,000 years and each sedimentary layer is produced every 2500 years which give us 200 layers. 

We can use the stratal mesh to retreive this information:

In [ ]:
LST1 = LST1/2500
TST1 = TST1/2500
HST1 = HST1/2500
FST1 = FST1/2500
LST2 = LST2/2500
TST2 = TST2/2500
print 'Number of layers recorded in the output :',strat.nz

We can now associate each layer to a given system tract by defining a specific color map:

- HST / STcolor[2]
- FST / STcolor[3]
- LST / STcolor[0]
- TST / STcolor[1]

In [ ]:
STcolors = []
for k in range(LST1[0],LST1[1]):
    STcolors.append(cLST)
for k in range(TST1[0],TST1[1]):
    STcolors.append(cTST)
for k in range(HST1[0],HST1[1]):
    STcolors.append(cHST)
for k in range(FST1[0],FST1[1]):
    STcolors.append(cFST)
for k in range(LST2[0],LST2[1]):
    STcolors.append(cLST)
for k in range(TST2[0],TST2[1]):
    STcolors.append(cTST)

Again we use **plotly** to visualise the cross-section and the associated system tracts:


- LST = yellow
- TST = green
- HST = blue
- FST = purple

In [ ]:
#help(strata.viewSectionST)

In [ ]:
strata.viewSectionST(width = 1000, height = 600, cs=strat, colors=STcolors,
                   rangeX=[2000,18000], rangeY=[-1500,500], 
                   linesize=0.5, title='system tract cross-section')